In [2]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
from io import StringIO
from chromatopy.readers import ShimadzuReader

In [3]:
path_dir = "/Users/max/Documents/training_course/martina"
path_file = "/Users/max/Documents/training_course/martina/Output-sample 0.txt"

In [20]:
sr = ShimadzuReader(path=path_dir)

sections = sr.read_file(path_file)
sections

{'timestamp': datetime.datetime(2023, 12, 12, 11, 15, 12),
 'injection_volume': 20.0,
 'injection_volume_unit': 'µL',
 'Chromatograms': [{'retention_times': [0.0,
    0.00833,
    0.01667,
    0.025,
    0.03333,
    0.04167,
    0.05,
    0.05833,
    0.06667,
    0.075,
    0.08333,
    0.09167,
    0.1,
    0.10833,
    0.11667,
    0.125,
    0.13333,
    0.14167,
    0.15,
    0.15833,
    0.16667,
    0.175,
    0.18333,
    0.19167,
    0.2,
    0.20833,
    0.21667,
    0.225,
    0.23333,
    0.24167,
    0.25,
    0.25833,
    0.26667,
    0.275,
    0.28333,
    0.29167,
    0.3,
    0.30833,
    0.31667,
    0.325,
    0.33333,
    0.34167,
    0.35,
    0.35833,
    0.36667,
    0.375,
    0.38333,
    0.39167,
    0.4,
    0.40833,
    0.41667,
    0.425,
    0.43333,
    0.44167,
    0.45,
    0.45833,
    0.46667,
    0.475,
    0.48333,
    0.49167,
    0.5,
    0.50833,
    0.51667,
    0.525,
    0.53333,
    0.54167,
    0.55,
    0.55833,
    0.56667,
    0.575,
  

In [21]:
from chromatopy.core import Measurement

In [22]:
c = Measurement(**sections)
print(c)

Measurement
├── id = 7cb60ce9-1084-41f2-b01a-18bddf87bf14
├── Chromatograms
│   └── 0
│       └── Chromatogram
│           ├── id = 0becd88c-d412-418b-85d1-a4d0946d8e24
│           ├── peaks
│           │   ├── 0
│           │   │   └── Peak
│           │   │       ├── id = 0e1230f9-3410-41c3-be74-47f012129e5f
│           │   │       ├── retention_time = 0.143
│           │   │       ├── retention_time_unit
│           │   │       │   └── Unit
│           │   │       │       ├── id = 719090b5-b9c4-460e-bc9b-e8bd94d62a04
│           │   │       │       ├── name = min
│           │   │       │       └── bases
│           │   │       │           └── 0
│           │   │       │               └── BaseUnit
│           │   │       │                   ├── scale = 60.0
│           │   │       │                   ├── kind = s
│           │   │       │                   └── exponent = 1.0
│           │   │       ├── peak_start = 0.017
│           │   │       ├── peak_end = 0.342
│           │   │

In [40]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(x=c.retention_times, y=c.signals, mode="lines"))
for peak in c.peaks:
    fig.add_vline(x=peak.retention_time, line_width=1, line_color="red")
fig.show()

In [ ]:
pd.read_csv(StringIO(sections[0]["LC Chromatogram(Detector A-Ch1)"]), skiprows=0).head(
    10
)

,Interval(msec),500
0,# of Points,1801
1,Start Time(min),0.000
2,End Time(min),15.000
3,Intensity Units,mV
4,Intensity Multiplier,0.001
5,Wavelength(nm),215
6,R.Time (min),Intensity
7,0.00000,-132
8,0.00833,-132
9,0.01667,-52


In [ ]:
from io import StringIO
from tokenize import String
import matplotlib.pyplot as plt
import pandas as pd

tab = sections["LC Chromatogram(Detector A-Ch1)"]

pd.read_csv(StringIO(tab), skiprows=8).plot(x="R.Time (min)")
plt.vlines(8.06, -20000, 400000, color="red")

TypeError: 'ShimadzuReader' object is not subscriptable

In [ ]:
import pandas as pd


def parse_meta(self, sections: dict, section_name: str, nrows: int) -> dict:
    """Parse the metadata in a section as keys-values."""
    meta_table = pd.read_table(
        StringIO(sections[section_name]), nrows=nrows, header=None
    )
    meta = {row[0]: row[1] for _, row in meta_table.iterrows()}

    return meta


def parse_table(
    self, sections: dict, section_name: str, skiprows: int = 1
) -> Optional[pd.DataFrame]:
    """Parse the data in a section as a table."""
    table_str = sections[section_name]

    # Count number of non-empty lines
    num_lines = len([l for l in re.split("[\r\n]+", table_str) if len(l)])

    if num_lines <= 1:
        return None

    return pd.read_table(StringIO(table_str), header=1, skiprows=skiprows)


def get_compound_table(sections: dict, detector: str = "A"):
    section_name = f"Compound Results(Detector {detector})"
    meta = parse_meta(sections, section_name, 1)
    table = parse_table(sections, section_name, skiprows=1)

    assert (
        table is None or int(meta["# of IDs"]) == table.shape[0]
    ), "Declared number of compounds and table size differ"

    return table

In [ ]:
sections.get_compound_table(sections, detector="B")
compounds[["Name", "Conc."]].to_dict()

AttributeError: 'dict' object has no attribute 'get_compound_table'

In [ ]:
with open(path, "r", encoding="ISO-8859-1") as f:
    print(f.read())

[Header]
Application Name,LCsolution
Version,1.25
Data File Name,C:\LabSolutions\Data\aldolaze\Martina\aldolna adicija\batch_21022012\EtAc\0.lcd
Output Date,12.12.2023.
Output Time,11:15:12

[File Information]
Type,Data File
Generated,21.2.2012. 13:16:32
Generated by,Admin
Modified,21.2.2012. 13:33:38
Modified by,Admin

[Sample Information]
Operator Name,Admin
Acquired,21.2.2012. 13:16:57
Sample Type,0:Unknown
Level,0
Sample Name,0
Sample ID,
ISTD Amount 1,1
ISTD Amount 2,1
ISTD Amount 3,1
ISTD Amount 4,1
ISTD Amount 5,1
ISTD Amount 6,1
ISTD Amount 7,1
ISTD Amount 8,1
ISTD Amount 9,1
ISTD Amount 10,1
ISTD Amount 11,1
ISTD Amount 12,1
ISTD Amount 13,1
ISTD Amount 14,1
ISTD Amount 15,1
ISTD Amount 16,1
ISTD Amount 17,1
ISTD Amount 18,1
ISTD Amount 19,1
ISTD Amount 20,1
ISTD Amount 21,1
ISTD Amount 22,1
ISTD Amount 23,1
ISTD Amount 24,1
ISTD Amount 25,1
ISTD Amount 26,1
ISTD Amount 27,1
ISTD Amount 28,1
ISTD Amount 29,1
ISTD Amount 30,1
ISTD Amount 31,1
ISTD Amount 32,1
Sample Amount,1
Di